In [1]:
!pip install faiss-cpu requests

  Using cached faiss_cpu-1.12.0-cp311-cp311-win_amd64.whl.metadata (5.2 kB)
Using cached faiss_cpu-1.12.0-cp311-cp311-win_amd64.whl (18.2 MB)



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import faiss
import numpy as np
import requests

In [4]:
dataset=[
"I go by the name Uzumaki-AK online", 
"though most people who know me in closer circles sometimes call me pookie.",
"I am a full-stack developer who has spent the last several years experimenting with technology in every possible way",
"For me, coding is not just about solving tasks or earning a living; it is about exploring "
"creativity through logic and building things that others can actually use.",
"My main tools revolve around JavaScript and its ecosystem. ",
"I-ve gone deep with Node.js on the backend and React on the frontend",
"and I-ve learned to appreciate how these technologies empower rapid prototyping while still being powerful enough for production-grade systems."
]

In [ ]:

import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer api key"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

In [6]:
generate_embeddings("my name is sophhia")

array([ 0.03796376, -0.04532918, -0.04571564, ...,  0.01877728,
       -0.01573109,  0.00168223], shape=(1536,))

In [7]:
embeddings=[]
for i in dataset:
  emb = generate_embeddings(i)
  embeddings.append(emb)


In [8]:
embeddings

[array([ 0.05650667, -0.00188265, -0.04055153, ..., -0.01368648,
        -0.00550236, -0.01260294], shape=(1536,)),
 array([ 0.01651739, -0.06606957, -0.04036649, ..., -0.03424269,
         0.02018324,  0.0164893 ], shape=(1536,)),
 array([-0.03051191, -0.05097594,  0.00287364, ..., -0.04148042,
        -0.01182337, -0.01362515], shape=(1536,)),
 array([ 0.02763267, -0.00928965,  0.01249812, ..., -0.01862905,
        -0.03024422, -0.0056055 ], shape=(1536,)),
 array([-0.02506366,  0.03327802,  0.01847903, ..., -0.02696939,
        -0.04284611,  0.01028438], shape=(1536,)),
 array([-0.03849872, -0.01082376,  0.0028983 , ..., -0.01797975,
        -0.01297825,  0.02212202], shape=(1536,)),
 array([ 0.05689325, -0.01038343, -0.00823419, ..., -0.01509944,
        -0.00492135,  0.0049898 ], shape=(1536,))]

In [9]:
embeddings[0]

array([ 0.05650667, -0.00188265, -0.04055153, ..., -0.01368648,
       -0.00550236, -0.01260294], shape=(1536,))

In [10]:
# converting this entire embedding to embedding array 
emb_arr=np.vstack(embeddings)

In [11]:
emb_arr

array([[ 0.05650667, -0.00188265, -0.04055153, ..., -0.01368648,
        -0.00550236, -0.01260294],
       [ 0.01651739, -0.06606957, -0.04036649, ..., -0.03424269,
         0.02018324,  0.0164893 ],
       [-0.03051191, -0.05097594,  0.00287364, ..., -0.04148042,
        -0.01182337, -0.01362515],
       ...,
       [-0.02506366,  0.03327802,  0.01847903, ..., -0.02696939,
        -0.04284611,  0.01028438],
       [-0.03849872, -0.01082376,  0.0028983 , ..., -0.01797975,
        -0.01297825,  0.02212202],
       [ 0.05689325, -0.01038343, -0.00823419, ..., -0.01509944,
        -0.00492135,  0.0049898 ]], shape=(7, 1536))

In [13]:
# we took dimensio coz faiss neds it to make indexes 
dimension=emb_arr.shape[1]
index = faiss.IndexFlatL2(dimension)

In [14]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000021B7C012A60> >

In [17]:
# storing data indide the inddex 

index.add(emb_arr)

In [18]:
index.ntotal

14

In [20]:
# to store 1 record lts say a text data 
text2 = 'her nam is shrerya-ji,she is a good girl'

In [21]:
generate_embeddings(text2)

array([ 0.02481547, -0.01420271, -0.05133616, ..., -0.00677601,
        0.05061818, -0.03439613], shape=(1536,))

In [23]:
# reshaping to do insertion
newtext =generate_embeddings(text2).reshape(1,-1)

In [24]:
newtext

array([[ 0.0248236 , -0.01422981, -0.05121832, ..., -0.00675579,
         0.05058988, -0.03438496]], shape=(1, 1536))

In [25]:
index.add(newtext)

In [26]:
index.ntotal

15

In [28]:
# ots storing in the memory so ot store in the physical form 
faiss.write_index(index,'xyz.faiss')

In [29]:
# now even we will create emb for the query 
query = 'who is uzumaki-ak'

In [31]:
query_vec = generate_embeddings(query).reshape(1,-1)

In [32]:
query_vec

array([[ 0.05445392, -0.00295541,  0.01044114, ..., -0.02443214,
        -0.01567579, -0.01285843]], shape=(1, 1536))

In [34]:
distance,indices=index.search(query_vec,2)

In [35]:
distance

array([[0.66178054, 0.66178054]], dtype=float32)

In [36]:
indices

array([[0, 7]])

In [37]:
dataset[0]

'I go by the name Uzumaki-AK online'